In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [60]:
df = pd.read_csv('data/Pokemon_cleaned.csv')

In [61]:
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,HasTwoTypes,Type_combined
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0,1,Grass / Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0,1,Grass / Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0,1,Grass / Poison
3,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,0,0,Fire /
4,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,0,0,Fire /


### Selección de Features

In [62]:
# 1. Features numéricas
num_features = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'HasTwoTypes'] 
# 2. Features categóricas
cat_features = ['Type 1', 'Type 2']

X = df[num_features + cat_features]
y = df['Legendary']

### OneHotEncoder para columnas categoricas

In [63]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='if_binary', handle_unknown='ignore'), cat_features)
    ], remainder='passthrough'
)

### Pipeline

In [64]:
pipeline = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=200))
])

### Train y Test

In [65]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

pipeline.fit(X_train, y_train)

c:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('pre',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['Type 1', 'Type 2'])])),
                ('clf',
                 RandomForestClassifier(class_weight='balanced',
                                        n_estimators=200, random_state=42))])

### Predicciones y evaluacion

In [66]:
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, digits=3))
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0      0.970     0.985     0.977       132
           1      0.667     0.500     0.571         8

    accuracy                          0.957       140
   macro avg      0.818     0.742     0.774       140
weighted avg      0.953     0.957     0.954       140

Matriz de confusión:
[[130   2]
 [  4   4]]


### Probemos diferentes modelos

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

In [68]:
modelos = {
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=5000, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=7),
    "XGBoost": XGBClassifier(scale_pos_weight=(len(y_train)-sum(y_train))/sum(y_train), random_state=42, eval_metric='logloss')
}

resultados = {}

# Creamos variable para guardar el pipeline de la Logistic Regression
pipeline_logreg = None

for nombre, modelo in modelos.items():
    pipeline = Pipeline([
        ('pre', preprocessor),
        ('clf', modelo)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    resultados[nombre] = {
        "accuracy": report['accuracy'],
        "f1_legendary": report['1']['f1-score'],
        "recall_legendary": report['1']['recall'],
        "precision_legendary": report['1']['precision']
    }
    # Si es Logistic Regression, lo guardamos para uso futuro
    if nombre == "Logistic Regression":
        pipeline_logreg = pipeline

print("\nResumen de desempeño de modelos:")
df_resultados = pd.DataFrame(resultados).T.round(3)
display(df_resultados)



Resumen de desempeño de modelos:


,accuracy,f1_legendary,recall_legendary,precision_legendary
Logistic Regression,0.979,0.842,1.000,0.727
Gradient Boosting,0.950,0.588,0.625,0.556
KNN,0.964,0.615,0.500,0.800
XGBoost,0.971,0.750,0.750,0.750


## Comparación de modelos de clasificación para predecir Pokémon legendarios

Se evaluaron varios modelos clásicos de clasificación:

- **Logistic Regression** obtuvo el mejor F1-score para la clase legendario (**0.84**) y recall perfecto (**1.0**), identificando correctamente **todos los legendarios** del set de prueba. Su precisión fue de 0.73, lo que significa que algunos no legendarios fueron clasificados erróneamente como legendarios.
- **Random Forest** y **KNN** tuvieron desempeños muy similares: ambos alcanzaron un F1-score de 0.78 para legendarios y un recall de 0.88 (anterior), pero en los resultados actuales, Random Forest no está en la tabla comparativa, y KNN bajó a F1 = 0.62 y recall = 0.50. En la matriz de confusión de Random Forest, el modelo clasificó correctamente 4 de 8 legendarios, pero también predijo 4 falsos positivos.
- **Gradient Boosting** tuvo el F1 más bajo para legendarios (**0.59**) y recall de 0.625. Fue el modelo más conservador, priorizando precisión (0.56) pero detectando menos legendarios.
- **XGBoost** mostró un desempeño medio, con F1-score de 0.75 y recall de 0.75, pero no superó a Logistic Regression.

**Conclusión:**  
Para este problema, **Logistic Regression sigue siendo la mejor opción si se prioriza no dejar fuera ningún legendario (recall = 1.0)**, aunque con algunos falsos positivos. **XGBoost** ofrece un buen balance entre precisión y recall, mientras que **KNN** y **Gradient Boosting** se quedan atrás en la detección de legendarios en este experimento.

Se recomienda, como siguientes pasos:
- Ajustar hiperparámetros, especialmente para Random Forest y XGBoost.
- Analizar los falsos positivos y negativos para entender mejor los errores de clasificación.
- Considerar técnicas adicionales de balanceo para optimizar la identificación de legendarios.




## Hagamos predicciones con el modelo de Regresión Logística para algunos pokemons legendarios

In [69]:
# Define los valores del Pokémon (Solgaleo)
nuevo_pokemon = pd.DataFrame([{
    'HP': 137,
    'Attack': 137,
    'Defense': 107,
    'Sp. Atk': 113,
    'Sp. Def': 89,
    'Speed': 97,
    'Generation': 7,
    'HasTwoTypes': 1,
    'Type 1': 'Psychic',
    'Type 2': 'Steel'
}])

prediccion = pipeline_logreg.predict(nuevo_pokemon)

print(f"Predicción: {'Legendario' if prediccion[0]==1 else 'No legendario'}")

Predicción: Legendario


In [70]:
# Define los valores del Pokémon (Tapu Fini)
nuevo_pokemon = pd.DataFrame([{
    'HP': 70,
    'Attack': 75,
    'Defense': 115,
    'Sp. Atk': 95,
    'Sp. Def': 130,
    'Speed': 85,
    'Generation': 7,
    'HasTwoTypes': 1,
    'Type 1': 'Water',
    'Type 2': 'Fairy'
}])

prediccion = pipeline_logreg.predict(nuevo_pokemon)

print(f"Predicción: {'Legendario' if prediccion[0]==1 else 'No legendario'}")

Predicción: Legendario


## Ahora probemos con pokemons no legendarios

In [71]:
# Define los valores del Pokémon (Dhelmise)
nuevo_pokemon = pd.DataFrame([{
    'HP': 70,
    'Attack': 131,
    'Defense': 100,
    'Sp. Atk': 86,
    'Sp. Def': 90,
    'Speed': 40,
    'Generation': 7,
    'HasTwoTypes': 1,
    'Type 1': 'Ghost',
    'Type 2': 'Grass'
}])

prediccion = pipeline_logreg.predict(nuevo_pokemon)

print(f"Predicción: {'Legendario' if prediccion[0]==1 else 'No legendario'}")

Predicción: No legendario


In [72]:
# Define los valores del Pokémon (Oranguru)
nuevo_pokemon = pd.DataFrame([{
    'HP': 90,
    'Attack': 60,
    'Defense': 80,
    'Sp. Atk': 90,
    'Sp. Def': 110,
    'Speed': 60,
    'Generation': 7,
    'HasTwoTypes': 1,
    'Type 1': 'Normal',
    'Type 2': 'Psychic'
}])

prediccion = pipeline_logreg.predict(nuevo_pokemon)

print(f"Predicción: {'Legendario' if prediccion[0]==1 else 'No legendario'}")

Predicción: No legendario


### Validación con ejemplos reales

Se realizaron pruebas adicionales usando el modelo de regresión logística entrenado, evaluando Pokémon de la 7ma generación que no fueron vistos por el modelo durante el entrenamiento:

- **Ejemplos usados:**
  - **Dos Pokémon legendarios** (Solgaleo y Tapu Fini)
  - **Dos Pokémon no legendarios** (Dhelmise y Oranguru)

El modelo **clasificó correctamente tanto a los legendarios como a los no legendarios** en estos casos, lo que demuestra su capacidad de generalización incluso con Pokémon de generaciones más recientes.

**Esto respalda la robustez del modelo y su utilidad para predecir correctamente nuevas instancias, incluso fuera del rango del entrenamiento original.**
